In [1]:
import os

In [2]:
%pwd

'/Users/satwik/Downloads/MLproj/airlines_sentiment_classification/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/Users/satwik/Downloads/MLproj/airlines_sentiment_classification'

In [ ]:
import torch
from flask import Flask
from flask import Flask, request, jsonify, render_template_string
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from pathlib import Path

# Initialize  the Flask app
app = Flask(__name__)


#Your SentimentInference class
class SentimentInference:
    def __init__(self, model_path: str):
        """
        Initializes the SentimentInference class.

        Args:
            model_path (str): Path to the trained model and tokenizer.
        """
        self.model_path = Path(model_path)
        self.model = AutoModelForSequenceClassification.from_pretrained(self.model_path)
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_path)
        self.sentiment_map = {0: 'negative', 1: 'neutral', 2: 'positive'}

    def predict_sentiment(self, text: str) -> str:
        """
        Predicts the sentiment of the input text.

        Args:
            text (str): Input text for sentiment prediction.

        Returns:
            str: Predicted sentiment ('negative', 'neutral', 'positive').
        """

        #Tokenize the input text 
        inputs = self.tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=128)

        # Get model predictions
        outputs = self.model(**inputs)
        predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
        predicted_class = torch.argmax(predictions, dim=1).item()

        # Map the predicted class to Sentiment
        return self.sentiment_map[predicted_class]


# Initialize the SentimentInference class
model_path = "/Users/satwik/Downloads/MLproj/airlines_sentiment_classification/artifacts/training/trained_model"
sentiment_inference = SentimentInference(model_path)

# Home page route
@app.route("/", methods=["GET"])
def home():
    return """
    <h1>Welcome to Sentiment Analysis / Test Classification</h1>
    <p>Click <a href="/predict">here</a> to go to the prediction page.</p>
    """

# Predict page route
def predict():
    if request.method == "GET":
        # Render a basic HTML form for text input
        return render_template_string('''
        <h1>Sentiment Prediction</h1>
        <form method="POST">
            <label for="text">Enter your text:</label><br>
            <textarea id="text" name="text" rows="4" cols="50"></textarea><br><br>
            <input type="submit" value="Predict">
        </form>
        ''')
    elif request.method == 'POST':
        # Get the input text from the form
        text = request.form.get("text")
         
        # Validate the input
        if not text:
            return jsonify({"error": " No text provided"}), 400
        
        # predict the sentiment
        try:
            sentiment = sentiment_inference.predict_sentiment(text)
            return jsonify({"sentiment": sentiment})

        except Exception as e:
            return jsonify({"error": str(e)}), 500

if __name__ == "__main__":
    # Run the flask app
    app.run(host="0.0.0.0", port=5000)





 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.24:5000
Press CTRL+C to quit
127.0.0.1 - - [11/Aug/2025 03:34:24] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Aug/2025 03:34:25] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [11/Aug/2025 03:34:26] "GET /predict HTTP/1.1" 404 -
192.168.1.24 - - [11/Aug/2025 03:34:44] "GET / HTTP/1.1" 200 -
192.168.1.24 - - [11/Aug/2025 03:34:44] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [11/Aug/2025 03:36:08] "GET /predict HTTP/1.1" 404 -


In [6]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from pathlib import Path
import logging

#Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class PredictionPipeline:
    def __init__(self):
        """
        Initializes the PredictionPipeline class.
        """
        self.model_path = Path("artifacts/training/trained_model") # Path to trained model
        self.sentiment_map = {0: "negative", 1: "neutral", 2: "positive"}
        self.model = None
        self.tokenizer = None

        #Debug: print the model path
        if not self.model_path.exists():
            raise FileNotFoundError(f"Model path does not exist: {self.model_path}")

        self.load_model()
    
    def load_model(self):
        """Loads the trained model and tokenizer
        """
        try:
            self.model = AutoModelForSequenceClassification.from_pretrained(self.model_path)
            self.tokenizer = AutoTokenizer.from_pretrained(self.model_path)

        except Exception as e:
            logger.error(f"Error loading model or tokenizer: {e}")
            raise e
    
    def predict(self, text: str) -> str:
        """ 
        Predicts the sentiment of the input text

        Args:
            text(str): Input text for Sentiment prediction.
        
        Returns:
            str: Predicted sentiment ('negative', 'neutral', 'positive').
        """
    
        try:
            #Tokenize the input text
            inputs = self.tokenizer(text, return_tensors='pt', paddding=True, truncation=True, max_length=128)

            #Get model predictions
            outputs = self.model(**inputs)
            predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
            predicted_class = torch.argmax(predictions, dim=1).item()


            #Map the predicted class to sentiment
            return self.sentiment_map[predicted_class]

        except Exception as e:
            logger.error(f"Error during prediction: {e}")
            raise e

In [7]:
import shutil

In [10]:
import shutil 
from pathlib import Path


#Define source and destination paths
source_dir = Path("/Users/satwik/Downloads/MLproj/airlines_sentiment_classification/artifacts/training/trained_model")
destination_dir = Path("model")

# Create the destination folder if it doesn't exist
destination_dir.mkdir(parents=True, exist_ok=True)

#Copy files
for file in source_dir.iterdir():
    if file.is_file():
        shutil.copy(file, destination_dir / file.name)
        print(f"Copied: {file.name}")

print("All files copied successfully.")

Copied: model.safetensors
Copied: tokenizer_config.json
Copied: special_tokens_map.json
Copied: config.json
Copied: tokenizer.json
Copied: training_args.bin
Copied: vocab.txt
All files copied successfully.
